In [1]:
import numpy as np
import mne

rng = np.random.RandomState(12345)

%load_ext rpy2.ipython

# Simulate sensor data
For this example, we'll simulate some EEG data for 10 subjects and 2 conditions. Each condition will consist of 100 trials. We will think of each trial as presenting the subject with a word (a **noun** in trial 1, and a **verb**
in trial 2).

In [3]:
fs = 1000
channels = range(32)
subjs = range(12)
conds = ['noun', 'verb']
nouns = ['time', 'year', 'people', 'way', 'day', 'man', 'thing', 'woman',
         'life', 'child', 'world', 'school', 'state', 'family', 'student',
         'group', 'country', 'problem', 'hand', 'part']
verbs = ['be', 'have', 'do', 'say', 'go', 'get', 'make', 'know', 'see', 
         'say', 'think', 'come', 'want', 'use', 'find', 'give', 'tell', 
         'work', 'call', 'try']
words = nouns + verbs
trials_per_word = 10
epoch_dur = 2.

#t = np.linspace(0, epoch_dur, int(epoch_dur * fs))
#data = np.empty((len(subjs), len(words) * trials_per_word, len(channels), len(t)))

global_intercept = 3
subj_intercepts = rng.randn(len(subjs)) * 5
cond_intercepts = np.repeat([13, 5], [len(nouns), len(verbs)])
                            
word_intercepts = rng.randn(len(words)) * 1.5
subj_noise = np.exp(1 + rng.randn(len(subjs)))

eps = rng.randn(len(subjs), len(words), trials_per_word)


alpha_powers = (global_intercept + 
                subj_intercepts[:, np.newaxis, np.newaxis] + 
                cond_intercepts[np.newaxis, :, np.newaxis] +
                word_intercepts[np.newaxis, :, np.newaxis] + 
                subj_noise[:, np.newaxis, np.newaxis] * eps)




In [4]:
alpha_powers.shape


(12, 40, 10)

First, we will consider the following **mixed-effects model**:

\begin{equation}
  \log y_{c,j,i} = \beta_0 + \beta_c + \beta_j + \epsilon_{c,j,i}
\end{equation}

$\epsilon_{c,j,i} \sim \mathcal{N}(0, \sigma_j)$

In [ ]:
%%R -o foo -i bar
foo <- data.frame(A=rnorm(4), B=rpois(4, 0.5))
print(bar)